Inference Routine from a trained model

In [1]:
import json
import os
import torch
from argparse import ArgumentParser

from pytorch_lightning import seed_everything

from transformers import EarlyStoppingCallback

from src.datasetComposer import DatasetBuilder, composed_train_path, composed_test_path, compactComposer, test_path, train_path, test_path,setupTokenizer
from src.inference_routine import InferenceGenerator
from src.datasetHandlers import SmartCollator
from src.model_utils import get_basic_model
from src.trainerArgs import CustomTrainer, getTrainingArguments
os.environ["WANDB_DISABLED"] = "true"
os.environ["TOKENIZERS_PARALLELISM"] = "false"



iterative_gen = True
composed_already = True

# Define the parameters used to set up the models
modeltype = 'iterative' if iterative_gen else 'normal'  # either baseline or 'earlyfusion'

# either t5-small,t5-base, t5-large, facebook/bart-base, or facebook/bart-large
modelbase = 't5-base' #'facebook/bart-base'

# we will use the above variables to set up the folder to save our model
pre_trained_model_name = modelbase.split(
    '/')[1] if 'bart' in modelbase else modelbase

# where the trained model will be saved
output_path = 'TrainModels/' + modeltype + '/'+pre_trained_model_name#+'/'

#tests = json.load(open(test_path,encoding='utf-8'))


rand_seed = 453
seed_everything(rand_seed)
device = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')

arguments = train_arguments = {'output_dir': output_path,
                               'warmup_ratio': 0.2,
                               #'disable_tqdm':False,
                               'per_device_train_batch_size': 8,
                               'num_train_epochs': 4,
                               'lr_scheduler_type': 'cosine',
                               'learning_rate': 5e-5,
                               'evaluation_strategy': 'steps',
                               'logging_steps': 500,
                               
                               'seed': rand_seed}

/home/james/CodingProjects/Local_level_model_explanations/env/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-01-04 10:35:52.375411: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-04 10:35:52.977939: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/usr/local/cuda/lib64:/home/james/Downloads/TensorRT-8.5.1.7/lib
2023-01-04 10:35:52.978003: W tensorflo

In [2]:
#load the modules from the inference routine
from types import SimpleNamespace
from src.inference_routine import NarratorUtils,ExplanationRecord,LocalLevelExplanationNarration

In [3]:

narrator_utils = NarratorUtils(modelbase,output_path)

# initialise the model
classification_explanator = narrator_utils.initialise_Model()




/home/james/CodingProjects/Local_level_model_explanations/env/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


 Dont forget to call initialise_Model() before running any inference


In [4]:
import copy

import random
# Example of input

ml_task_name = 'Car Insurance Risk'

prediction_probabilities = {'Low': 0.76, 'High': 0.24}
# the features used to make the prediction
feature_names = ['Height', 'Mar_status', 'cur_loc', 'nb_friends', 'last_trip']

bcc = feature_names.copy()
random.shuffle(bcc)

# get the order and directions of influence from the explainable output from the XAI technique
# the methods expects the keys ['explanation_order','positives','negatives','ignore']
# 'positives' is the list of all the features with positive influence on the prediction decision and 'negatives' is the inverse.
# 'ignore' is the list of features identified as having very limited contribution to the prediction decision

attributions = {'explanation_order': ['Height', 'last_trip', 'cur_loc', 'nb_friends', 'Mar_status','Income'],
                'positives': ['Height', 'last_trip', 'Mar_status'],
                'negatives': ['cur_loc', 'nb_friends'],
                'ignore': ['Income']
                
                 }


In [13]:
# if we want to generate the texts via the iterative generation then we have to define the style
# We want our output text to first table about the prediction output
# step 1: talk about the feature order based on the attributions
# step 2: talk about the features with positive contributions to the decision
# step 3: ----- negative features
# step 4: ------- features with limited influence
# step 5: Make conclusion based on all the input information

# this will instruct the narrator to follow our desired output style
iterative_generation_steps = {'step 0': '',
                              'step 1': attributions['explanation_order'],
                              'step 2': attributions['positives'][:2],
                              'step 3': attributions['negatives'][:1],
                              'step 4': attributions['negatives'][1:] + attributions['positives'][2:],
                              'step 5': attributions['ignore'],
                              'step 6': '-'
                              }


full_text_generation_steps = {'step 0': '',
                              'step 1': attributions['explanation_order'],
                              }




In [14]:
iterative_gen = True
generation_instruction = iterative_generation_steps if iterative_gen else full_text_generation_steps
# Process the explanation output and the text generation instruction
exp_record = ExplanationRecord(ml_task_name,feature_names,prediction_probabilities, attributions,iterative_mode=True)
processed = exp_record.setup_generation_steps(generation_instruction,)

# the final bit 
iterativeGen =LocalLevelExplanationNarration(classification_explanator,narrator_utils,device,iterative_mode=True,)
iterativeGen.generateTexts(processed)

Global seed set to 456


The ML model predicted the label : Low


["The model labels the given case as Low with a confidence level equal to 76.0%, while the other label,  High, has an uncertainty level of 24.10%. cur_loc, nb_friends, and Mar_status are shown to have a negative impact on the model's decision here. Height and last_trip have a positive impact on the model's decision, increasing the likelihood of the Low label."]

In [15]:
iterative_gen = False
generation_instruction = iterative_generation_steps if iterative_gen else full_text_generation_steps
# Process the explanation output and the text generation instruction
exp_record = ExplanationRecord(ml_task_name,feature_names,prediction_probabilities, attributions,iterative_mode=True)
processed = exp_record.setup_generation_steps(generation_instruction,)

# the final bit 
iterativeGen =LocalLevelExplanationNarration(classification_explanator,narrator_utils,device,iterative_mode=True,)
iterativeGen.generateTexts(processed)

Global seed set to 456


The ML model predicted the label : Low
All facts presented


["The model labels the given case as Low with a confidence level equal to 76.0%, while the other label,  High, has an uncertainty level of 24.10%. cur_loc, nb_friends, and Mar_status are shown to have a negative impact on the model's decision here."]